## 동아일보

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
from urllib.parse import urlparse

FASTAPI_URL = 'https://j11a301.p.ssafy.io/video'

def lambda_handler(event, context):
    
    return {
        'statusCode': 200,
        'body': 'donga-news-crawler from Lambda is do!'
    }

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

categories = {
    '정치': 'https://www.donga.com/news/Politics',
    '경제': 'https://www.donga.com/news/Economy',
    '국제': 'https://www.donga.com/news/Inter',
    '사회_사건': 'https://www.donga.com/news/Society/Event', # 앞 분기 기준
    '경제_과학': 'https://www.donga.com/news/It/List', # 앞 분기 기준
    '사회': 'https://www.donga.com/news/Society',
    '스포츠': 'https://www.donga.com/news/Sports',
}

result_categories = {
    '정치': 'politics',
    '경제': 'economy',
    '국제': 'world',
    '사회_사건': 'accident',
    '경제_과학': 'science',
    '사회': 'society',
    '스포츠': 'sports',
}

def scrap_article(article_url, category):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        page_category = soup.find('ol', class_='breadcrumb').find('li', class_='breadcrumb_item').find('a').get_text(strip=True)
        logging.info(f"page_category: {page_category}")
        
        logging.info(f"category: {category}")

        category_parts = category.split('_')
        
        # 카테고리가 아닌 글은 패스
        if page_category not in category_parts:
            logging.info(f"{article_url} 는 해당 카테고리가 아니어서 생략합니다. ")
            return None
    except AttributeError:
        logging.error(f"{article_url} 는 찾을 수 없습니다.")
        return None

    # 헤드라인 스크랩
    try:
        title = soup.find('section', {'class': 'head_group'}).find('h1').get_text(strip=True)
    except AttributeError:
        title = 'N/A'
        
    try:
        date_button = soup.find('button', {'aria-controls': 'dateInfo'})
        time_span = date_button.find('span', {'aria-hidden': 'true'})
        time = time_span.get_text(strip=True)  # "2024-09-30 21:44" 형태
        time = time.replace(" ", "T") + ":00+09:00"  # ISO 8601 형식으로 변환 (예: 2024-09-30T21:44:00+09:00)
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        # 광고, 이미지 등 제외하고 텍스트만 추출
        content_section = soup.find('section', class_='news_view')
        for tag in content_section(['figure', 'script', 'div', 'ul']):  # 불필요한 태그 제거
            tag.decompose()
        content = content_section.get_text(separator="\n", strip=True)
    except AttributeError:
        content = 'N/A'

    result_category = result_categories.get(category, 'unknown')

    return {'title': title, 'time': time, 'content': content, 'source_url': article_url, 'category': result_category}

def post_article_to_server(article_data):
    
    fastapi_url = 'https://your-fastapi-url.com/process_data'
    headers = {'Content-Type': 'application/json'}
    
    try:
        response = requests.post(FASTAPI_URL, json=article_data, headers=headers)
        response.raise_for_status()
        logging.info(f'\'statusCode\': 200 \n\'body\': \'요청 성공\'')
    except requests.exceptions.RequestException as e:
        logging.error(f'\'statusCode\': 500 \n\'Internal Server Error\': {e}')

def check_for_new_articles(base_url, category):
    response = requests.get(base_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_count = 0
    cards = soup.find_all('article', class_='news_card')
    cards.reverse()
    for card in cards:
        if article_count >= 2:
            break

        link_tag = card.find('a', href=True)
        if link_tag and 'href' in link_tag.attrs:
            article_link = link_tag['href']
        else:
            logging.warning("article_link is None or missing 'href'.")
            continue
    
        logging.debug(f"Scraping article: {article_link}")

        result = scrap_article(article_link, category)

        if result is None:
            continue
        
        # post_article_to_server(result) # 서버가 완성되면 주석을 풀어라...
        print(result)
        article_count += 1

def scrap_all_categories():
    
    for category, base_url in categories.items():
        logging.info(f"category (base_url): {category} ({base_url})")
        check_for_new_articles(base_url, category)
        time.sleep(2)

scrap_all_categories()

INFO:root:category (base_url): 정치 (https://www.donga.com/news/Politics)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.donga.com/news/Politics/article/all/20241008/130173324/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics/article/all/20241008/130173324/1 HTTP/11" 200 None
INFO:root:page_category: 정치
INFO:root:category: 정치
DEBUG:root:Scraping article: https://www.donga.com/news/Politics/article/all/20241008/130173341/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics/article/all/20241008/130173341/1 HTTP/11" 200 None
INFO:root:page_category: 정치


{'title': '북한, 이틀 연속 쓰레기 풍선 살포…26번째 도발', 'time': '2024-10-08T05:42:00+09:00', 'content': '북한이 이틀 연속으로 대남 쓰레기 풍선 살포에 나섰다.\n합동참모본부는 8일 오전 5시 9분쯤 국방부 출입기자단에 보낸 휴대전화 문자메시지를 통해 “현재 풍향을 고려할 때 대남 쓰레기 풍선이 경기북부 및 수도권 지역으로 이동 가능성이 있다”라면서 이처럼 밝혔다.\n합참은 “국민들께서는 적재물 낙하에 주의하시고, 떨어진 풍선을 발견하시면 접촉하지 마시고 가까운 군부대나 경찰에 신고해주시기 바란다”라고 당부했다.\n이번 쓰레기 풍선 부양은 북한이 지난 5월 28일 1차 ‘오물 풍선’을 살포한 이후 26번째 도발이다.\n(서울=뉴스1)', 'source_url': 'https://www.donga.com/news/Politics/article/all/20241008/130173324/1', 'category': 'politics'}


INFO:root:category: 정치


{'title': '北 김정은 “군사초강국, 핵강국 향한 발걸음 더 빨라질 것”', 'time': '2024-10-08T06:29:00+09:00', 'content': '김정은 북한 노동당 총비서가 “군사초강국, 핵강국으로 향한 발걸음이 더 빨라질 것”이라면서 군사력 강화 행보를 멈추지 않을 것임을 재확인했다. 동시에 우리를 향해 “소름이 끼친다”라며 대화 의사가 전혀 없음을 강조했다.\n노동당 기관지 노동신문은 8일 김 총비서가 전날인 7일 김정은국방종합대학을 창립 60주년을 맞아 한 연설에서 “조선반도(한반도)에서 전략적 힘의 균형의 파괴는 곧 전쟁을 의미한다”면서 “적을 항상 억제하고 정세를 관리할 수 있는 물리적 힘을 가져야 한다는 우리의 자위국방 건설 논리는 바늘 들어갈 틈도 없이 완벽하고 정당하다”면서 이같이 주장했다.\n이어 “적들이 우리 국가를 반대하는 무력 사용을 기도한다면 공화국 무력은 모든 공격력을 주저 없이 사용할 것”이라면서 여기에는 ‘핵무기 사용’이 배제되지 않는다고 재차 강조했다.\n그러면서 “한미 군사동맹이 괴뢰들 스스로가 광고하는 것처럼 핵동맹으로 완전히 변이된 현시점에서 우리 국가의 핵대응태세는 더더욱 한계를 모르는 높이에서 완비돼야 한다”라며 “세계 최대의 핵보유국과 그것을 공동으로 만지작거리려는 가장 간악한 괴뢰들이 우리 앞에 있는 환경하에서 우리의 견해와 선택, 결심은 결코 변할 수 없다”라고도 말했다.\n김 총비서는 이날 연설에서 남한을 ‘별개의 국가’로 대하겠다는 ‘남북 2국가론’을 재차 언급했다.\n그는 “대한민국은 의식하는 것조차도 소름이 끼치고 그 인간들과는 마주 서고 싶지도 않다”라며 “과거엔 우리가 그 무슨 ‘남녘 해방’이라는 소리도 많이 했고 ‘무력 통일’이라는 말도 했지만 지금은 전혀 이에 관심이 없으며 두 개 국가를 선언하면서부터는 더더욱 그 나라를 의식하지도 않는다”라고 말했다.\n(서울=뉴스1)', 'source_url': 'https://www.donga.com/news/Politics/articl

INFO:root:category (base_url): 경제 (https://www.donga.com/news/Economy)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Economy HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.donga.com/news/Economy/article/all/20241008/130173355/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Economy/article/all/20241008/130173355/1 HTTP/11" 200 None
INFO:root:page_category: 경제
INFO:root:category: 경제
DEBUG:root:Scraping article: https://www.donga.com/news/Inter/article/all/20241008/130173381/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Inter/article/all/20241008/130173381/1 HTTP/11" 200 None
INFO:root:page_category: 국제


...

{'title': '계란자조금관리위원회-대한산란계협회, 굿네이버스에 1000만원 기부', 'time': '2024-10-08T09:02:00+09:00', 'content': '글로벌 아동권리 전문 비정부기구(NGO) 굿네이버스는 계란자조금관리위원회, 대한산란계협회와 함께 국내외 위기가정 아동 지원을 위한 기부금 전달식을 진행했다고 밝혔다.\n기부금 전달식은 지난 2일 서울 양재동 AT센터에서 열린 ‘에그 페스티벌’ 행사 중 진행됐다. 전달식에는 김중곤 굿네이버스 사무총장, 안두영 계란자조금관리위원회 위원장 겸 대한산란계협회장 등이 참석했다.\n계란자조금관리위원회와 대한산란계협회는 이날 굿네이버스에 기부금 1000만 원을 전달했다. 기부금은 국내외 위기가정 아동을 돕는 데 사용될 예정이다. 앞서 지난 7월 3개 기관은 국내외 위기가정 아동 지원을 위해 △기부금 조성 사업 △물품(계란) 기부 사업 △해외지역개발사업 등을 협력하는 업무 협약식을 체결한 바 있다.\n안두영 계란자조금관리위원회 위원장 겸 대한산란계협회장은 “위기가정 아동을 비롯한 도움이 필요한 이웃을 지원할 수 있어 기쁘다”며 “앞으로도 각 지역 계란 농가들과 함께 현금, 물품(계란) 기부 등 다양한 나눔 활동을 꾸준히 이어가겠다”고 말했다.\n김중곤 굿네이버스 사무총장은 “전국 계란 농가의 따뜻한 마음을 모아 선한 영향력을 전해주신 계란자조금관리위원회와 대한산란계협회에 감사드린다”며 “두 기관과 함께 국내외 소외된 아동의 건강한 성장과 지역사회 자립을 지원하는 데 더욱 힘쓰겠다”고 전했다.', 'source_url': 'https://www.donga.com/news/Society/article/all/20241008/130173581/2', 'category': 'society'}
{'title': '‘밀양 성폭행’ 가해자들 신상 무단공개한 유튜버 구속', 'time': '2024-10-08T09:41:00+09:00', 'content': '지난 2004년 발생한 ‘경남 밀양 집단 성폭력 사건’

INFO:root:category (base_url): 스포츠 (https://www.donga.com/news/Sports)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Sports HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.donga.com/news/Sports/article/all/20241007/130169353/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Sports/article/all/20241007/130169353/1 HTTP/11" 200 None
INFO:root:page_category: 스포츠
INFO:root:category: 스포츠
DEBUG:root:Scraping article: https://www.donga.com/news/Sports/article/all/20241007/130171242/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Sports/article/all/20241007/130171242/1 HTTP/11" 200 None
INFO:root:page_category: 스포츠


{'title': '안세영, 올림픽 후 첫 공식 대회…9일 전국체전 단체전 16강 출격', 'time': '2024-10-07T16:57:00+09:00', 'content': '양궁 김우진·임시현 등 파리올림픽 메달리스트 40명 참가\n2024 파리 올림픽에서 금메달을 획득한 후 폭탄 발언을 한 배드민턴 간판 안세영(삼성생명)이 올림픽 이후 첫 공식 대회에 모습을 드러낸다.\n11일 시작하는 제105회 전국체육대회에서다.\n올해 전국체육대회는 11일부터 17일까지 경남 김해종합운동장 등 75개 경기장에서 벌어진다. 47개 정식 종목, 2개 시범 종목 경기가 열린다.\n선수 2만여명, 임원 1만여명이 참가하는 이번 대회에 올해 파리 올림픽에서 메달을 딴 스포츠 스타 46명 중 40명이 참가한다.\n이 중 배드민턴과 하키 등은 사전경기로 대회 개막 전에 치러진다.\n파리 올림픽 배드민턴 여자 단식에서 1996년 애틀랜타 대회 방수현 이후 28년 만에 금메달을 목에 건 안세영은 9일 경남 밀양 배드민턴 경기장에서 열리는 여자 일반부 단체전 16강전에 부산 대표로 나선다.\n부산의 단체전 16강전 상대는 인천 대표인 인천국제공항이다.\n안세영은 파리 올림픽 단식 금메달 획득 직후 인터뷰 등을 통해 대한배드민턴협회의 문제점을 지적해 논란을 일으켰다. 부상 관리, 대표팀 훈련과 운영 방식 등에 불만을 드러냈다.\n8~9월 일본오픈, 코리아오픈에 모두 불참한 안세영에게 전국체전은 올림픽 이후 첫 공식 대회다.\n사전경기로 진행된 태권도에서는 파리 올림픽 여자 67㎏ 초과급에서 동메달을 딴 이다빈(서울시청)이 여자 일반부 73㎏급에서 동메달을 수확했다.\n파리 올림픽 금메달리스트 중 가장 먼저 체전 일정을 소화한 태권도 김유진(울산광역시체육회)은 7일 벌어진 여자 57㎏급에서 금메달을 획득했다.\n양궁 남녀 3관왕에 등극한 김우진(청주시청)과 임시현(임실군청)은 12일부터 체전에 나서고, 펜싱 남자 사브르 2관왕 오상욱은 개막일인 11일 개인전 첫 경기를 치른다.\n파리 

INFO:root:category: 스포츠


{'title': '백승호, ‘英 3부’ 버밍엄과 재계약…2028년까지 뛴다', 'time': '2024-10-07T18:51:00+09:00', 'content': '“축구 경력에 좋은 일이라고 판단”\n국가대표 출신 미드필더 백승호(27)가 잉글랜드 리그1(3부리그) 버밍엄과 계약 기간을 2년 연장했다.\n버밍엄은 7일(한국시간) “백승호와 재계약을 체결했다. 백승호는 버밍엄에 2028년 6월 30일까지 머문다”고 밝혔다.\n백승호는 지난 1월 잉글랜드 챔피언십(2부리그) 소속의 버밍엄과 2026년 6월 30일까지 2년 6개월 계약을 맺은 바 있다.\n이번 계약 연장은 예상치 못한 소식이다.\n지난 1월 겨울 이적 시장을 통해 버밍엄 유니폼을 입은 백승호는 빠르게 팀의 주축으로 자리 잡았다. 하지만 팀 강등을 막지 못하며 올 시즌 3부리그에서 뛰고 있다.\n여름 이적 시장에서 리즈, 셰필드, 선덜랜드 등 챔피언십의 여러팀이 백승호에게 관심을 보였다. 하지만 올 시즌 바로 승격에 도전하는 버밍엄은 팀의 핵심인 백승호를 잔류시켰다.\n일각에서는 지난 여름에 팀을 옮기지 못한 백승호가 겨울 이적 시장 때 상위 리그 팀으로 이적을 추진할 것이라고 전망했다.\n그러나 백승호는 자신에게 높은 신뢰를 보이는 팀에 남기로 결심했다. 백승호는 올 시즌 버밍엄이 치른 경기에 모두 선발 출전하는 등 팀 내에서 핵심으로 입지를 다졌다.\n크리스 데이비스 버밍엄 감독은 공식적으로 “백승호는 올 시즌은 물론 앞으로 계속해서 버밍엄의 핵심 선수가 될 것이라고 확신한다”며 백승호에게 힘을 불어 넣었다.\n재계약을 체결한 백승호는 “계약을 연장해 정말 기쁘다. 새 시즌이 시작된 이후 버밍엄은 성과를 낼 수 있을 것이라는 믿음이 생겼다”면서 “모든 사람들이 버밍엄이 좋은 과정을 보내고 있는 것을 지켜보고 있을 것이다. 버밍엄에서 생활하는 것이 축구 경력에 좋은 경험이라고 생각한다”고 밝혔다.\n한편 버밍엄은 9경기를 치른 현재 7승 1무 1패(승점 22)로 1경기를 더 치른 렉섬(승점 2

## 중앙일보

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
from urllib.parse import urlparse

FASTAPI_URL = 'https://j11a301.p.ssafy.io/video'

def lambda_handler(event, context):
    
    return {
        'statusCode': 200,
        'body': 'joongang-news-crawler from Lambda is do!'
    }

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

categories = {
    '정치': 'https://www.joongang.co.kr/politics',
    '경제': 'https://www.joongang.co.kr/money',
    '국제': 'https://www.joongang.co.kr/world',
    '사회_사건': 'https://www.joongang.co.kr/society/accident', # 앞 분기 기준
    '경제_과학': 'https://www.joongang.co.kr/money/science', # 앞 분기 기준
    '사회': 'https://www.joongang.co.kr/society',
    '스포츠': 'https://www.joongang.co.kr/sports',
}


result_categories = {
    '정치': 'politics',
    '경제': 'economy',
    '국제': 'world',
    '사회_사건': 'accident',
    '경제_과학': 'science',
    '사회': 'society',
    '스포츠': 'sports',
}

def scrap_article(article_url, category):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:

        page_category = soup.find('a', {'class': 'title'}).get_text(strip=True)
        logging.info(f"page_category: {page_category}")
        
        logging.info(f"category: {category}")

        category_parts = category.split('_')
        
        # 카테고리가 아닌 글은 패스
        if page_category not in category_parts:
            logging.info(f"{article_url} 는 해당 카테고리가 아니어서 생략합니다. ")
            return None
    except AttributeError:
        logging.error(f"{article_url} 는 찾을 수 없습니다.")
        return None

    # 헤드라인 스크랩
    try:
        title = soup.find('h1', {'class': 'headline'}).get_text(strip=True)
    except AttributeError:
        title = 'N/A'

    # 생성일 스크랩
    try:
        date = soup.find('p', {'class': 'date'})
        time = date.find('time').get('datetime')
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        content = soup.find('div', {'class': 'article_body'}).get_text(strip=True)
    except AttributeError:
        content = 'N/A'

    result_category = result_categories.get(category, 'unknown')

    return {'title': title, 'source_url': article_url, 'content': content, 'category': result_category, 'source_created_at': time, "press_id": 1}

def post_article_to_server(article_data):
    
    headers = {'Content-Type': 'application/json'}
    
    try:
        response = requests.post(FASTAPI_URL, json=article_data, headers=headers)
        response.raise_for_status()
        logging.info(f'\'statusCode\': 200 \n\'body\': \'요청 성공\'')
    except requests.exceptions.RequestException as e:
        logging.error(f'\'statusCode\': 500 \n\'Internal Server Error\': {e}')

def check_for_new_articles(base_url, category):
    response = requests.get(base_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_count = 0
    cards = soup.find_all('li', class_='card')
    cards.reverse()
    for card in cards:
        if article_count >= 2:
            break

        link_tag = card.find('a', href=True)
        if link_tag and 'href' in link_tag.attrs:
            article_link = link_tag['href']
        else:
            logging.warning("article_link is None or missing 'href'.")
            continue
    
        logging.debug(f"Scraping article: {article_link}")

        result = scrap_article(article_link, category)

        if result is None:
            continue

        # post_article_to_server(result) # 서버가 완성되면 주석을 풀어라...
        print(result)
        article_count += 1

def scrap_all_categories():
    
    for category, base_url in categories.items():
        logging.info(f"category (base_url): {category} ({base_url})")
        check_for_new_articles(base_url, category)
        time.sleep(2)

scrap_all_categories()

INFO:root:category (base_url): 정치 (https://www.joongang.co.kr/politics)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /politics HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282595
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282595 HTTP/11" 200 None
INFO:root:page_category: 정치
INFO:root:category: 정치
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282602
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282602 HTTP/11" 200 None


{'title': "한동훈·이재명 또 '부산행'…금정구청장 보선에 사활 거는 이유", 'source_url': 'https://www.joongang.co.kr/article/25282595', 'content': '한동훈 국민의힘 대표가 5일 오후 부산 금정구 이마트 금정점 앞에서 윤일현 금정구청장 후보 지원유세를 하고 있다.(왼쪽 사진) 이재명 더불어민주당 대표가 3일 오후 부산 금정구 부산대역 앞에서 김경지 금정구청장 후보 지원유세를 하고 있다. 뉴스110·16 기초단체장 재·보선은 인천 강화군수, 부산 금정구청장, 전남 영광·곡성군수를 뽑는 선거다. 그중에서도 가장 치열한 곳은 부산 금정구청장 보궐선거다.\xa0이재명 더불어민주당 대표는 9일 부산 금정구를 찾아 김경지 민주당 금정구청장 후보 지원유세에 나선다. 같은 날 한동훈 국민의힘 대표도 부산 금정구에서 현장 최고위원회의를 열고, 윤일현 국민의힘 후보를 지원사격한다.두 사람이 금정구를 방문하는 것은 이번이 각각 네 번째(이 대표), 다섯 번째(한 대표)다.\xa0정치권에서는 “기초단체장 선거에 양당 대표가 이처럼 공을 들이는 건 이례적”이라는 말이 나온다. 예컨대 지난해 4월 창녕군수 보궐선거 당시 이 대표는 창녕군에 반나절만 머물렀다.민주당·조국혁신당은 6일 야권 단일후보로 김경지 민주당 후보를 선출하면서 윤일현 국민의힘 후보와의 일대일 구도를 형성하는 데 성공했다. 한국사회여론연구소(KSOI)의 가상대결 여론조사(10월 1~2일) 결과 윤일현 43.5%-김경지 40.0%로 오차범위(±4.4%포인트) 내 박빙이다. 부산 금정구청장 선거는 1995년 민선 이후 9번의 선거(재·보선 포함)에서 국민의힘 계열이 8번 승리할 만큼 보수가 강세였다. 하지만 최근 지지율 변화가 감지되자 양당 대표 모두 발벗고 나선 것이다.신재민 기자양당 대표의 등장은 두 사람이 처한 정치적 상황과도 무관치 않다는 평가다. 이 대표는 11월 15일(공직선거법 위반), 25일(위증교사) 1심 재판을 앞두고 있다. 재·보선에 승

INFO:root:page_category: 정치
INFO:root:category: 정치


{'title': '동행명령 엄포 놓고 때린 野…직무정지 이진숙 “일하고 싶다”', 'source_url': 'https://www.joongang.co.kr/article/25282602', 'content': '국회 과학기술정보방송통신위원회의 7일 방송통신위원회 국정감사는 탄핵소추로 직무가 정지된 이진숙 방송통신위원장의 청문회를 방불케 했다.당초 이날 오전 과방위의 방통위 국감에 이진숙 위원장은 출석하지 않았다. 탄핵 소추안 가결로 직무정지라 불출석하겠다는 사유서를 냈지만, 민주당 소속 최민희 과방위원장이 불출석 시 동행명령장 발부를 예고하면서 이날 오후 3시께 이 위원장은 모습을 드러냈다.\xa0이 위원장이 국회에 출석한 것은 지난 8월 14일 과방위의 ‘방송장악 관련 2차 청문회’ 이후 두 달 만이다.7일 국회 과학기술정보방송통신위원회에서 열린 방송통신위원회·방송통신사무소·시청자미디어재단·한국방송광고진흥공사 국정감사에서 최민희 위원장이 윤원일 대검찰청 형사 3과 과장, 강필구 방송통신위원회 혁신기획담당관 증인 추가 출석 요구의 건을 상정하고 있다. 뉴스1더불어민주당 등 야권은 지난 청문회 때부터 제기한 법인 카드 유용 의혹 등을 꺼내 들며 이 위원장에 대한 질의를 이어갔다. 이해민 조국혁신당 의원이 “세금으로 월급 따박따박 받아가고 있는데, (직무 정지된 이 위원장이) 하나도 불쌍하지 않다”고 하자, 이 위원장은 “원해서 뛰쳐나온 게 아니다. 나는 일을 하고 싶다”고 말했다.황정아 더불어민주당 의원은 이 위원장의 국감 불출석 사유서를 방통위 직원이 대리로 국회 행정실에 제출한 것을 들며\xa0“이 위원장은 법인카드도 관용차도 사적으로 마구잡이로 이용하더니 이제는 방통위마저 사유화하려 한다”고 꼬집었다. 이 위원장은 이에 대해 “방통위에 국회 출입 직원이 있는데 그쪽에서 먼저 사유서를 (자신이) 내면 되겠냐고 해 그렇게 하자고 답변했다”며 “공직자를 개인 비서로 부린 적 없다”고 반박했다.여당 의원들도 거들었다. 박정훈 국민의힘 의원은 “야당 과방위 위원

INFO:root:category (base_url): 경제 (https://www.joongang.co.kr/money)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /money HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282656
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282656 HTTP/11" 200 None
INFO:root:page_category: 경제
INFO:root:category: 경제
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282664
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282664 HTTP/11" 200 None


...

{'title': '진천 ‘2만t 쓰레기산’ 5년째 방치…안 치우나 못 치우나', 'source_url': 'https://www.joongang.co.kr/article/25282673', 'content': '충북 진천군 문백면 사양리 한 공터에 방치 폐기물이 5년 넘게 쌓여 있다. 프리랜서 김성태지난 3일 충북 진천군 문백면 사양리의 한 야산. 회색 철판으로 된 울타리 너머로 높이 7m쯤 되는 거대한 언덕이 보였다. 덩굴 식물과 잡초가 뒤엉켜 온통 초록빛이었다. 풀을 걷어내자 진짜 모습이 드러났다. 플라스틱병·의자·소파 내장재·폐건축자재 등 온갖 생활 폐기물이 겹겹이 쌓여있었다.이곳은 ‘진천 쓰레기산’이란 오명이 붙은 현장이다. 폐기물 중간재활용업체인 A사가 2018년부터 폐합성수지류 등 폐기물 2만3000t을 반입한 뒤 5년 넘게 방치 상태다. 허가받은 폐기물 보관용량(1200t)의 약 19배를 초과한 양이다. A사는 2020년 사업 허가가 취소된 뒤에도 쓰레기를 치우지 않았다. 쓰레기산 문제가 마을 밖으로 알려진 2021년 이후에도 마땅한 해결 방안을 찾지 못하면서 주변이 흉물처럼 변했다. 그 사이 업주가 구속됐고 땅 주인도 바뀌었다. 진천군은 A사가 맡긴 방치폐기물이행보증금 3억5000여만원으로 1380t을 치웠으나 나머지 폐기물은 여전히 처리되지 않고 있다.사업장 인근에서 만난 문모(87)씨는 “A사가 느닷없이 높은 벽을 치더니 밤낮으로 드나들며 고의로 쓰레기를 쌓았다”며 “진천군이 허가를 해줬으면 관리·감독을 철저하게 해야 했는데 그러지 못했다”고 지적했다.주민 권모(43)씨는 “쓰레기산이 오랫동안 방치되다 보니 비가 많이 오면 야적장에서 나온 물이 인근 논으로 흘러 들어가기도 한다”고 토로했다.사양리에 쓰레기산이 생긴 건 2018년 말부터다. 이 자리에 문을 연 A사는 폐합성수지류를 들여와 선별·파쇄·압축·포장 등 처리 과정을 거쳐 해외에 되파는 일을 했다. 사업장 규모는 1만250㎡다.쓰레기산이 처리되지 않은 가장 큰 이유는 땅 주인이 바뀌어

INFO:root:category (base_url): 스포츠 (https://www.joongang.co.kr/sports)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /sports HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282253
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282253 HTTP/11" 200 None
INFO:root:page_category: 스포츠
INFO:root:category: 스포츠
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25282254
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25282254 HTTP/11" 200 None


{'title': '하루 쉬고 나와 4이닝 1실점… 최고 잠수함 위용 뽐낸 KT 고영표', 'source_url': 'https://www.joongang.co.kr/article/25282253', 'content': '5일 열린 준플레이오프 1차전 역투로 승리를 이끈 KT 위즈 고영표. 뉴스1국내 최고 잠수함 고영표(33·KT 위즈)가 기대에 부응했다. 연이은 불펜 투구 이후 선발로 나섰지만, 자신의 역할을 톡톡히 해냈다.KT는 5일 서울 잠실구장에서 열린 준플레이오프(PO·5전 3승제) 1차전에서 LG 트윈스를 3-2로 이겼다. 준PO 1차전에서 승리한 KT는 유리한 고지를 점령했다.고영표는 KBO리그에서 가장 안정적인 선발투수다. 최근 4시즌 동안 가장 많은 퀄리티 스타트(QS·6이닝 이상 3자책점 이하 투구, 72회)를 기록했다. 7이닝 이상 던지는 퀄리티 스타트 플러스도 1위(42회)다. 다만 올 시즌엔 주춤했다. 부상과 부진이 겹쳐 18경기에 나가 6승 8패 평균자책점 4.95에 머물렀다.하지만 중요한 순간 원래의 모습이 나타났다. 선발투수로 등판한 그는 4회 1사까지 단 한 명의 타자도 1루에 내보내지 않았다. 4회에 안타 3개를 맞고 1점을 내주긴 했지만 LG 타선을 초반에 봉쇄했다. 이날 경기 최우수선수상(MVP)도 고영표에게 돌아갔다.고영표는 최근 본래 보직이 아닌 구원투수로 두 번이나 나섰다. 지난 1일 SSG 랜더스와의 1위 결정전에서 1과 3분의 2이닝(무실점)을 던졌고, 3일 두산과의 와일드카드 결정 2차전에서도 1이닝(무실점)을 투구했다. 그래서 이강철 감독도 50개 정도만 던지게 하려 했다.최근 두 차례 구원 등판 이후 하루만에 나왔지만 승리를 이끈 고영표. 뉴스1고영표는 경기 뒤 "감독님이 50개까지 생각하고 올렸는데 불편하면 빨리 말하고 했다. 불편하지 않았고, 100개까지 던질 수 있으니까 편하게 생각하시라고 했다. 4회 때 조금 지친 모습이 나와서 끊어가신 것 같다. 한 구 한 구 온 힘을 다해 던졌다"고 했다.그는 "물론

INFO:root:page_category: 스포츠
INFO:root:category: 스포츠


{'title': '준PO 1차전 잡은 KT 이강철 "우리 야구 했다"', 'source_url': 'https://www.joongang.co.kr/article/25282254', 'content': '준PO 1차전 승리 뒤 환하게 미소짓는 이강철 KT 위즈 감독(왼쪽 둘째). 연합뉴스KT의 마법이 또다시 가을을 수놓았다. 이강철 KT 위즈 감독은 "우리 야구를 했다"며 만족스러워했다.KT는 5일 서울 잠실구장에서 열린 준플레이오프(PO) 1차전에서 정규시즌 3위 LG를 3-2로 이겼다. 역대 준PO 1차전 승리 팀은 87.9%(33회 중 29차례) 확률로 PO에 진출했다.이강철 감독은 경기 뒤 "우리 패턴대로 KT 야구를 했다. 고영표는 6회까지 던질 수 있다고 했다. 4회에 마운드에 올라 체크를 했는데, 조금씩 떨어지고 있어 교체했다. 다음 투수들이 너무 잘 던져줘서 승리할 수 있었다"고 말했다.마운드 방문 뒤 고민스러운 표정의 이강철 감독. 뉴스1이강철 감독은 이날 문상철을 5번 타자·1루수로 기용했고,\xa0문상철은 2회 초 선제 결승 투런포를 터트렸다. 이 감독은 "타격코치가 기대해도 좋다고 말했다. 홈런을 쳐서 분위기를 잘 가져온 거 같다. 4회 말 실점으로 2-1이 된 뒤 5회 초 달아나는 추가점을 뽑은 게 (좋은)영향을 끼쳤다"고 말했다.4회 수비에선 삐걱대는 모습이 있었다. 1사 1·3루에서 1루주자 신민재가 2루 도루를 했고, 포수 장성우의 송구를 내야수들이 잡지 못해 중견수 앞까지 굴러가 실점했다. 이 감독은 고민하다 "사인 미스였다. 그때 분위기를 내줬다고 생각했는데 더 실점하지 않아서 이길 수 있다고 생각했다"고 말했다.반대로 9회 2사 1루에선 LG 대주자 김대현의 도루 시도를 간파하고 2루에서 잡아내 경기를 마무리했다. 이 감독은 "코치들이 \'간다\'고 이야기를 하더라. (송구 준비를 하기 위해)장성우 엉덩이가 올라가 있더라"며 웃었다.이날 KT는 성공적으로 투수진을 운용했다. 하루 휴식 후 나선 선발 고영표가 4이닝 1실점했고